In [ ]:
import torch

if torch.cuda.is_available():
    print("CUDA está disponible. Usando GPU.")
else:
    print("CUDA no está disponible. Usando CPU.")

CUDA no está disponible. Usando CPU.


In [ ]:
!pip install nltk
!pip install numba

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# navegar en google drive
%cd /content/drive/MyDrive/big data/
# mostrar el contenido
!ls


/content/drive/MyDrive/big data
large_text_file.txt  word_counter.ipynb


In [ ]:
import nltk
import random
from numba import cuda
import numpy as np

# Descargar el corpus de palabras de nltk
nltk.download('words')
from nltk.corpus import words

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


In [ ]:
import nltk
import random
from numba import cuda
import numpy as np

# corpus de palabras de nltk
nltk.download('words')
from nltk.corpus import words

# palabras en inglés de nltk
word_list = words.words()

# Filtrar palabras que solo contienen letras
word_list = [word for word in word_list if word.isalpha()]

@cuda.jit
def generate_words_gpu(word_indices, block_size, output):
    pos = cuda.grid(1)
    if pos < len(output):
        word_idx = word_indices[pos % block_size]
        output[pos] = word_idx

def generate_large_text_file(filename, word_list, target_size_gb=10):

    target_size_bytes = target_size_gb * (1024 ** 3)

    block_size = 1000
    word_indices = np.array([random.randint(0, len(word_list) - 1) for _ in range(block_size)], dtype=np.int32)
    word_indices_gpu = cuda.to_device(word_indices)

    word_len_avg = sum(len(word) for word in word_list) / len(word_list)
    approx_num_words = target_size_bytes // int(word_len_avg + 1)

    with open(filename, 'w') as f:
        words_per_write = 1000000
        output_gpu = cuda.device_array(words_per_write, dtype=np.int32)

        for _ in range(approx_num_words // words_per_write):
            generate_words_gpu[(words_per_write + block_size - 1) // block_size, block_size](word_indices_gpu, block_size, output_gpu)
            output_host = output_gpu.copy_to_host()
            words_to_write = ' '.join(word_list[idx] for idx in output_host)
            f.write(words_to_write + ' ')

    print(f"Archivo {filename} generado exitosamente.")

# Ruta en Google Drive donde se guardará el archivo
file_path = '/content/drive/MyDrive/big data/large_text_file.txt'

# Generar archivo de 10 GB en Google Drive
generate_large_text_file(file_path, word_list)


[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


Archivo /content/drive/MyDrive/big data/large_text_file.txt generado exitosamente.


In [ ]:
import os
from collections import Counter
from concurrent.futures import ProcessPoolExecutor
import re


In [ ]:
def process_chunk(chunk):
    words = re.findall(r'\b\w+\b', chunk.decode('utf-8').lower())
    return len(words), Counter(words)

def word_count(file_path, chunk_size=64 * 1024 * 1024):
    file_size = os.path.getsize(file_path)
    total_words = 0
    combined_counter = Counter()

    with open(file_path, 'rb') as f, ProcessPoolExecutor() as executor:
        futures = []
        for _ in range(0, file_size, chunk_size):
            chunk = f.read(chunk_size)
            futures.append(executor.submit(process_chunk, chunk))

        for future in futures:
            word_count, word_counter = future.result()
            total_words += word_count
            combined_counter.update(word_counter)

    return total_words, combined_counter

file_path = '/content/drive/MyDrive/big data/large_text_file.txt'

total_words, word_counter = word_count(file_path)

with open('/content/drive/MyDrive/big data/word_count_result.txt', 'w') as result_file:
    result_file.write(f'Total words: {total_words}\n')
    result_file.write('Top 10 most common words:\n')
    for word, count in word_counter.most_common(10):
        result_file.write(f'{word}: {count}\n')

print(f'Total words: {total_words}')
print(f'Los resultados han sido guardados en /content/drive/MyDrive/big data/word_count_result.txt')


Total words: 1073000133
Los resultados han sido guardados en /content/drive/MyDrive/big data/word_count_result.txt


[github](https://github.com/HarryLexvb/Big-Data)

In [ ]:
github: https://github.com/HarryLexvb/Big-Data